In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/36114_ADSI/AT2

/content/gdrive/MyDrive/36114_ADSI/AT2


In [3]:
# !pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.0 MB/s eta 0:00:00


In [11]:
import pandas as pd
import numpy as np
# import category_encoders as ce
import tensorflow as tf
import matplotlib.pyplot as plt
from joblib import dump
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [5]:
!pwd

/content/gdrive/MyDrive/36114_ADSI/AT2


In [33]:
# Read in the csv
df = pd.read_csv("/content/gdrive/MyDrive/36114_ADSI/AT2/beer_reviews.csv")


In [34]:
# Drop unnecessary columns
df = df.drop(['brewery_id', 'review_time', 'review_overall', 'review_profilename', 'beer_beerid', 'beer_name'], axis=1)

# Check for missing values
df.isnull().sum()


brewery_name            15
review_aroma             0
review_appearance        0
beer_style               0
review_palate            0
review_taste             0
beer_abv             67785
dtype: int64

In [35]:
# Drop rows with missing 'brewery_name'
df = df.dropna(subset=['brewery_name'])

# Fill missing 'beer_abv' with median
df['beer_abv'] = df['beer_abv'].fillna(df['beer_abv'].median())

# Check if there are still any missing values
df.isnull().sum()


<ipython-input-35-e82ba2250631>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['beer_abv'] = df['beer_abv'].fillna(df['beer_abv'].median())


brewery_name         0
review_aroma         0
review_appearance    0
beer_style           0
review_palate        0
review_taste         0
beer_abv             0
dtype: int64

In [36]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le_brewery = LabelEncoder()
# le_beer = LabelEncoder()

# Fit and transform 'brewery_name' and 'beer_name'
df['brewery_name'] = le_brewery.fit_transform(df['brewery_name'])
# df['beer_name'] = le_beer.fit_transform(df['beer_name'])

# Display the first few rows of the DataFrame
df.head()


,brewery_name,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_abv
0,5438,2.0,2.5,Hefeweizen,1.5,1.5,5.0
1,5438,2.5,3.0,English Strong Ale,3.0,3.0,6.2
2,5438,2.5,3.0,Foreign / Export Stout,3.0,3.0,6.5
3,5438,3.0,3.5,German Pilsener,2.5,3.0,5.0
4,1480,4.5,4.0,American Double / Imperial IPA,4.0,4.5,7.7


In [37]:
# Initialize LabelEncoder for the target variable
le_beer_style = LabelEncoder()

# Fit and transform 'beer_style'
df['beer_style'] = le_beer_style.fit_transform(df['beer_style'])

# Display the first few rows of the DataFrame
df.head()


,brewery_name,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_abv
0,5438,2.0,2.5,65,1.5,1.5,5.0
1,5438,2.5,3.0,51,3.0,3.0,6.2
2,5438,2.5,3.0,59,3.0,3.0,6.5
3,5438,3.0,3.5,61,2.5,3.0,5.0
4,1480,4.5,4.0,9,4.0,4.5,7.7


In [38]:
from sklearn.model_selection import train_test_split

# Separate features and target variable
X = df.drop('beer_style', axis=1)
y = df['beer_style']

# Perform the split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shape of the train/test data
X_train.shape, X_test.shape


((1269279, 6), (317320, 6))

In [39]:
# Calculate the number of unique classes in 'beer_style'
num_classes = df['beer_style'].nunique()
num_classes


104

In [29]:
# Initialize the constructor
model = Sequential()

# Input Layer
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))

# Hidden Layers
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.3)) # Dropout layer after each Dense layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

# Output Layer
model.add(Dense(104, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
# Define the model
model = Sequential()

# Add layers to the model
model.add(Dense(128, activation='relu', input_dim=6))  # Input layer
model.add(Dense(100, activation='relu'))  # Hidden layer
model.add(Dense(num_classes, activation='softmax'))  # Output layer

# Define the optimizer
optimizer = Adam(0.001)

In [24]:
# Define the model
model = Sequential()

# Add layers
model.add(Dense(128, input_dim=7, activation='relu'))  # Input layer
model.add(Dense(128, activation='relu'))  # Hidden layer
model.add(Dense(128, activation='relu'))  # Hidden layer
model.add(Dense(num_classes, activation='softmax'))  # Output layer

In [43]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


# Display the model summary
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 128)               896       
                                                                 
 dense_19 (Dense)            (None, 100)               12900     
                                                                 
 dense_20 (Dense)            (None, 104)               10504     
                                                                 
Total params: 24,300
Trainable params: 24,300
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=256)


Epoch 1/5
4959/4959 [==============================] - 26s 5ms/step - loss: 4.0982 - accuracy: 0.0815 - val_loss: 4.0957 - val_accuracy: 0.0829
Epoch 2/5
4959/4959 [==============================] - 26s 5ms/step - loss: 4.1119 - accuracy: 0.0812 - val_loss: 4.1267 - val_accuracy: 0.0785
Epoch 3/5
4959/4959 [==============================] - 27s 5ms/step - loss: 4.0929 - accuracy: 0.0832 - val_loss: 4.1176 - val_accuracy: 0.0805
Epoch 4/5
4959/4959 [==============================] - 26s 5ms/step - loss: 4.0460 - accuracy: 0.0897 - val_loss: 4.0104 - val_accuracy: 0.0942
Epoch 5/5
4959/4959 [==============================] - 26s 5ms/step - loss: 3.7536 - accuracy: 0.1237 - val_loss: 3.6455 - val_accuracy: 0.1332


In [47]:
# Save the model
model.save('beer_style_predictor.h5')

# Save the encoders
import pickle
with open('le_brewery.pkl', 'wb') as f:
    pickle.dump(le_brewery, f)
# with open('le_beer.pkl', 'wb') as f:
#     pickle.dump(le_beer, f)
with open('le_beer_style.pkl', 'wb') as f:
    pickle.dump(le_beer_style, f)